In [ ]:
!apt install pv
!cd /content; [ -d chroma-instance ] && (cd chroma-instance && git reset HEAD^ --hard && git pull -f) || git clone https://github.com/owen8877/chroma-instance/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pv is already the newest version (1.6.6-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
HEAD is now at 9d75922 Fixed another bug.
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0
Unpacking objects: 100% (7/7), done.
From https://github.com/owen8877/chroma-instance
   966f915..796506b  master     -> origin/master
Updating 9d75922..796506b
Fast-forward
 .../source/chroma_instance/model/fusion.py         | 124 +++++++++++++--------
 1 file changed, 76 insertions(+), 48 deletions(-)


In [ ]:
# Copy the backed up weights into working directory
from google.colab import drive
drive.mount('/content/drive')

!ln -s '/content/drive/MyDrive/colab_tmp/weights' '/content/chroma-instance/chroma_instance/weights'
!ln -s '/content/drive/MyDrive/colab_tmp/logs' '/content/chroma-instance/chroma_instance/logs'
!ln -s '/content/drive/MyDrive/colab_tmp/result' '/content/chroma-instance/chroma_instance/result'

Mounted at /content/drive


In [ ]:
!cd /content/chroma-instance/chroma_instance/source && ./download_model.sh

In [ ]:
!unzip /content/drive/MyDrive/colab_tmp/dataset/val2017_bbox.zip -d /content/chroma-instance/chroma_instance/dataset/

In [ ]:
import sys
sys.path.append('/content/chroma-instance/chroma_instance/source')

%load_ext tensorboard

%tensorflow_version 1.x
import tensorflow as tf

%tensorboard --logdir /content/chroma-instance/chroma_instance/logs

## Chroma Instance

In [ ]:
cd /content/chroma-instance/chroma_instance/source/chroma_instance

/content/chroma-instance/chroma_instance/source/chroma_instance


In [ ]:
from chroma_instance.config.FirstTest import FirstTestConfig
from chroma_instance.data.generator import Data
from chroma_instance.data.bbox_extract import extract_bbox
from mrcnn import utils, coco
import os

# strategy = tf.distribute.get_strategy()
# config = FirstTestConfig('fusion_huber_instance', ROOT_DIR='../../')
# train_data = Data(config.TRAIN_DIR, config)
# test_data = Data(config.TEST_DIR, config)

# config.COCO_MODEL_PATH = os.path.join(config.ROOT_DIR, "weights/mask_rcnn/coco.h5")
# if not os.path.exists(config.COCO_MODEL_PATH):
#     utils.download_trained_weights(config.COCO_MODEL_PATH)
# extract_bbox(config, config.TRAIN_DIR, strategy)
# extract_bbox(config, config.TEST_DIR, strategy)


Running on colab!


Using TensorFlow backend.


In [ ]:
# !rm ../../weights/chroma_instance/fusion_2obj -r
# !rm ../../logs/chroma_instance/fusion_2obj -r
# !rm ../../result/chroma_instance/fusion_2obj -r

from chroma_instance.model.fusion_with_instance import FusionModel
from datetime import datetime
from chroma_instance.util import prepare_logger
from tensorflow.python.keras.losses import huber_loss

config = FirstTestConfig('fusion_2obj_huber_instance', ROOT_DIR='../../')
config.NUM_EPOCHS = 10
train_data = Data(config.TRAIN_DIR, config)
test_data = Data(config.TEST_DIR, config)

with prepare_logger(train_data.batch_size, config) as logger:
    logger.write(str(datetime.now()) + "\n")

    print("Initializing Model...")
    colorizationModel = FusionModel(config, load_weight_path='../../weights/chroma_gan/imagenet.h5',
                                            ab_loss=lambda y_true, y_pred: tf.reduce_mean(huber_loss(y_true, y_pred, delta=0.05)))
    print("Model Initialized!")
    print("Start training")
    colorizationModel.train(train_data, test_data, logger, config=config)


Initializing Model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading chroma GAN parameter to instance network...
Layer fg_input_img not found in chroma gan.
Layer fg_pack_input not found in chroma gan.
Layer fg_input_tile not found in chroma gan.
model 3 skip
Successfully set weights for layer fg_conv2d_6
Successfully set weights for layer fg_batch_normalization_1
Successfully set weights for layer fg_conv2d_7
Successfully set weights for layer fg_batch_normalization_2
Successfully set weights for layer fg_conv2d_8
Successfully set weights for layer fg_batch_normalization_3
Successfully set weights for layer fg_conv2d_9
Successfully set weights for layer fg_batch_normalization_4
Successfully set weights for layer fg_flatten_1
Successfully set weights for layer fg_dense_1
Successfully set weights for layer fg_conv2d_10
Successfully set weights for layer

  0%|          | 0/500 [00:00<?, ?it/s]

batch_size=10 * total_batch=500



  0%|          | 1/500 [01:40<13:58:00, 100.76s/it]

[Epoch 0] [Batch 0/500] [generator loss: 0.003793] [discriminator loss: 2.820996]


...

[Epoch 6] [Batch 480/500] [generator loss: 0.015104] [discriminator loss: 0.002624]


 98%|█████████▊| 491/500 [18:05<00:19,  2.19s/it]

[Epoch 6] [Batch 490/500] [generator loss: 0.015093] [discriminator loss: 0.000160]


100%|██████████| 500/500 [18:24<00:00,  2.21s/it]


Saving models...
Models saved.
Sampling test images...


In [ ]:
cp /content/chroma-instance/chroma_instance/weights/chroma_instance/FirstTest/my_model_colorizationEpoch4.h5 /content/drive/MyDrive/colab_tmp/

In [ ]:
!cd /content/chroma-instance/chroma_instance/dataset && zip -r train_bbox.zip train_bbox/

In [ ]:
!cp /content/chroma-instance/chroma_instance/dataset/train_bbox.zip /content/drive/MyDrive/colab_tmp/dataset/